# Content-Enriched Recommender Systems

## Introduction

- Content-Based Recommendation systems suggest items that have similar features to items that a user has interacted with. 

- This results in a limited ability to expand on the users' existing interests. To address this shortcoming, content-enriched (hybrid) models have been proposed and leverage both content-features and user-item interactions to make recommendations. 

- One such approach is the Two-Tower network which uses content-aware features to enhance the collaborative filtering (CF) approaches. 

- Two-Tower Networks are often leveraged in the retrieval or candidate generation stage of recommender systems in which the initial set of candidates is selected out of all the possible candidates.


Retrieval models often composed of two sub-models:

1. A query model computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
2. A candidate model computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give us a query-candidate affinity score - the higher the score, the better the match between the candidate and the query. The dot product of a user, item pair that actually interacted is high and the dot product of a non-interacting pair is close to 0.

The following notebook implements this two-tower model using [Tensorflow Recommenders](https://www.tensorflow.org/recommenders).

![](https://miro.medium.com/max/1400/0*bii7baVk5nF6lulx)



## The Dataset

The Movielens dataset is a popular dataset from the [GroupLens](https://grouplens.org/datasets/movielens/) research group at the University of Minnesota. It contains a set of ratings given to movies by a set of users, and is often used for recommender system research.

In this demo, the data will be treated as an implicit feedback: expressesing which movies the users watched (and rated), and which they did not. Thus, a users' watches gives information about which things they prefer to see and which they'd rather not see. Every movie a user has watched is a positive example and every movie they have not seen is a negative example.

This model predicts a set of movies from the catalogue that the user is likely to watch. 

## Imports

In [1]:
from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

## Loading the Data

We load the MovieLens dataset from [Tensorflow Datasets](https://www.tensorflow.org/datasets). Loading `movielens/100k_ratings` yields a `tf.data.Dataset` object containing the ratings data and loading `movielens/100k_movies` yields a `tf.data.Dataset` object containing only the movies data.

<!-- the MovieLens dataset does not have predefined splits, all data are under `train` split. -->

In [2]:
# Ratings
ratings = tfds.load("movielens/100k-ratings", split="train", data_dir="../data")
# Features of movies
movies = tfds.load("movielens/100k-movies", split="train", data_dir="../data")

2022-09-21 11:03:11.669133: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling ../data/movielens/100k-movies/0.1.1.incompleteV0JI3T/movielens-train.tfrecord*...:   0%|          | …

Dataset movielens downloaded and prepared to ../data/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


The ratings dataset returns a dictionary of movie id, user id, rating, timestamp, movie information, and user information.

In [3]:
ratings_data = []
for x in ratings.take(5).as_numpy_iterator():
  ratings_data.append(x)

pd.DataFrame.from_dict(ratings_data)

2022-09-21 11:04:01.562475: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,[7],b'357',"b""One Flew Over the Cuckoo's Nest (1975)""",46.0,879024327,True,b'138',4,b'doctor',4.0,b'53211'
1,25.0,"[4, 14]",b'709',b'Strictly Ballroom (1992)',32.0,875654590,True,b'92',5,b'entertainment',2.0,b'80525'
2,18.0,[4],b'412',"b'Very Brady Sequel, A (1996)'",24.0,882075110,True,b'301',17,b'student',4.0,b'55439'
3,50.0,"[5, 7]",b'56',b'Pulp Fiction (1994)',50.0,883326919,True,b'60',4,b'healthcare',4.0,b'06472'
4,50.0,"[10, 16]",b'895',b'Scream 2 (1997)',55.0,891409199,True,b'197',18,b'technician',3.0,b'75094'


The movies dataset contains the movie id, movie title, and data on what genres it belongs to.

In [4]:
movies_data = []
for x in movies.take(5).as_numpy_iterator():
  movies_data.append(x)


pd.DataFrame.from_dict(movies_data)

2022-09-21 11:04:01.643019: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


,movie_genres,movie_id,movie_title
0,[4],b'1681',b'You So Crazy (1994)'
1,"[4, 7]",b'1457',b'Love Is All There Is (1996)'
2,"[1, 3]",b'500',b'Fly Away Home (1996)'
3,[0],b'838',b'In the Line of Duty 2 (1987)'
4,[7],b'1648',"b'Niagara, Niagara (1997)'"


In this example, we will explore the `user_id` and `movie_title` fields in the dataset.

In [5]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"]
})
movies = movies.map(lambda x: x["movie_title"])

The `user_id` and `movie_title` features are categorical and we need to map them to embedding vectors in order to use them in a deep learning model. Our goal is to predict which user is going to watch which movie. To do that, we represent each user and each movie by an embedding vector. Initially, these embeddings will take on random values - but during training, we will adjust them so that embeddings of users and the movies they watch end up closer together.

To turn the raw categorical features into embeddings, we must first translate the values into a range of continuous integers, also known as a mapping or vocabulary. For example, we map the value "Star Wars" to 15. Then, we need to convert these integers into embeddings.

In [6]:
unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

## Model Definition

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

### The Query Tower

We start by defining the UserModel which will convert the raw input to feature embeddings using `Embedding` layers.

An embedding layer has two dimensions: the first dimension tells us how many distinct categories we can embed; the second tells us how large the vector representing each of them can be.




In [7]:
class UserModel(tf.keras.Model):
  
  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.

    return self.user_embedding(inputs["user_id"])

### The Candidate Tower

We can do the same with the movie tower.

In this case, our `movie_title` feature is text, so the first transformations we should apply is tokenization and create a vocabulary of these tokens. The `tf.keras.layers.TextVectorization` layer from Keras performs both these steps. To embed the text, we can average all of the words' embeddings to get a single embedding.

In [8]:
class MovieModel(tf.keras.Model):
  
  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(movies)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)

### The Full Model

We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines building models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

### Metrics

In our training data we have positive (user, movie) pairs. We need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates. If the score for the positive pair is higher than for all other candidates, our model is highly accurate.

We use the `tfrs.metrics.FactorizedTopK` which has one required argument: the dataset of candidates that are used as implicit negatives for evaluation. In this case, that's the `movies` dataset, converted into embeddings via our movie model. This metric measures how good the model is at picking the true candidate out of all possible candidates in the system.

As the model trains, the loss is falling and a set of top-k retrieval metrics is updated. These tell us whether the true positive is in the top-k retrieved items from the entire candidate set. For example, a top-5 categorical accuracy metric of 0.2 would tell us that, on average, the true positive is in the top 5 retrieved items 20% of the time.

In [9]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
    })
    movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

## Fitting and evaluating

After defining the model, we use standard Keras fitting and evaluation routines to fit and evaluate the model.

In [10]:
# Spslit the data into a training and a test set.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [12]:
baseline_model = MovielensModel()
baseline_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

baseline_model.fit(cached_train, epochs=3)

baseline_train_accuracy = baseline_model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
baseline_test_accuracy = baseline_model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {baseline_train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {baseline_test_accuracy:.2f}.")

Epoch 1/3


40/40 [==============================] - 16s 271ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0092 - factorized_top_k/top_5_categorical_accuracy: 0.0172 - factorized_top_k/top_10_categorical_accuracy: 0.0256 - factorized_top_k/top_50_categorical_accuracy: 0.0824 - factorized_top_k/top_100_categorical_accuracy: 0.1473 - loss: 14579.4626 - regularization_loss: 0.0000e+00 - total_loss: 14579.4626
Epoch 2/3
40/40 [==============================] - 12s 270ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0020 - factorized_top_k/top_5_categorical_accuracy: 0.0126 - factorized_top_k/top_10_categorical_accuracy: 0.0251 - factorized_top_k/top_50_categorical_accuracy: 0.1129 - factorized_top_k/top_100_categorical_accuracy: 0.2133 - loss: 14136.2136 - regularization_loss: 0.0000e+00 - total_loss: 14136.2136
Epoch 3/3
40/40 [==============================] - 13s 275ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0021 - factorized_top_k/top_5_categorical_accuracy: 0.0155 

5/5 [==============================] - 4s 413ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0077 - factorized_top_k/top_10_categorical_accuracy: 0.0185 - factorized_top_k/top_50_categorical_accuracy: 0.1049 - factorized_top_k/top_100_categorical_accuracy: 0.2132 - loss: 30995.8991 - regularization_loss: 0.0000e+00 - total_loss: 30995.8991
Top-100 accuracy (train): 0.29.
Top-100 accuracy (test): 0.21.


## Results in Literature

Two-Tower Networks use content-aware features to enhance the collaborative filtering (CF) approaches.

In a study conducted on e-commerce data from e-bay, this approach was shown to have improved results over an existing Recently-Viewed-Item (RVI) method. The existing method recommends items that a user has recently viewed ranked by the viewed item’s recency; RVI is widely used as a way of generating personalized recommendations in production systems. It is also a difficult baseline to beat in terms of generating user engagement, given that these are items the user has engaged with recently. It has also been shown to outperform CF-based
methods.

This method outperformed the RVI method in several Recall@k metrics that were
measured. This shows that the model is able to generate appropriate personalized recommendations based on the user’s current shopping mission, and potentially inspire new ones given the right user history.

In an online evaluation, initial A/B test results showed that compared to the current personalized recommendation module in production, the proposed method increases the conversion rate by ∼6% to generate recommendations for 90% of listing page impressions.

Read more about this study [here](https://arxiv.org/pdf/2102.06156.pdf).


For more information about Tensorflow Recommenders, read more [here](https://www.tensorflow.org/recommenders/examples/basic_retrieval) and for more information about Two-Tower Models, read [here](https://research.google/pubs/pub48840/).